In [1]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# будем отображать графики прямо в jupyter'e
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append('//')

import data_load as dl

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 12, 9

In [3]:
# footbal stat without api
# http://wildstat.ru/p/50/ch/EUR_CC_1988_1989

# дорогой и мощный api
# https://footystats.org/download-stats-csv

# каталог всего
# https://www.jokecamp.com/blog/guide-to-football-and-soccer-data-and-apis/

In [4]:
# хелп тут: https://www.football-data.org/documentation/quickstart
# полный список соревнований (за все годы)
url='http://api.football-data.org/v2/competitions'
# параметры конкретного соревнования (за все годы) 
# в примере испания
# url='http://api.football-data.org/v2/competitions/2014'
# спиоск матчей конкретного соревнования, тут уже есть фильтры
# в примере испания
# url='http://api.football-data.org/v2/competitions/2014/matches'
# данные по конкретному матчу
# севилья - райо вельйкано
# url='http://api.football-data.org/v2/matches/331356'
# данные команды
# url='http://api.football-data.org/v2/teams/78'

In [5]:
import requests
headers = {
    'X-Auth-Token': '44ce50aab7504f17ae93e674300009f6',
}

params = (
#     ('matchday', '1'),
#     ('dateFrom','2021-08-14'),
#     ('dateTo','2021-08-16'),
#     ('stage','REGULAR_SEASON'),
#     ('status','FINISHED'),
#     ('season','2016'),
)

r = requests.get(url, 
                 params=params, headers=headers)

print(r.status_code)

200


In [6]:
r.json().keys()

dict_keys(['count', 'filters', 'competitions'])

In [7]:
l1=[]
l2=[]
for i in range(r.json()['count']):
    l1.append(r.json()['competitions'][i]['name'])
    l2.append(r.json()['competitions'][i]['id'])
    
tournaments_table=pd.DataFrame([l1, l2]).T
tournaments_table.columns=['Name', 'ID']

In [8]:
tournaments_table.sample(3)

,Name,ID
156,Summer Olympics,2153
75,NB I,2128
125,ABSA Premiership,2083


In [9]:
tournaments_table[tournaments_table['ID']==2014]

,Name,ID
132,Primera Division,2014


In [10]:
# на вход - id турнире, на выходе - json
def get_league_data(id):
    url='http://api.football-data.org/v2/competitions/'+str(id)+'/matches'
    import requests
    headers = {
        'X-Auth-Token': '44ce50aab7504f17ae93e674300009f6',
    }

    r = requests.get(url, headers=headers)

    print(r.status_code)
    return r.json()

# когда получил запрос со всей лигой, её можно перевести в готовый датафрейм
def get_league_match(an):
    match_id_lst=[]
    home_lst=[]
    away_lst=[]
    h_score_lst=[]
    aw_score_lst=[]
    status_lst=[]
    date_lst=[]

    for i in range(an['count']):
        match_id_lst.append(an['matches'][i]['id'])
        home_lst.append(an['matches'][i]['homeTeam']['name'])
        away_lst.append(an['matches'][i]['awayTeam']['name'])
        h_score_lst.append(an['matches'][i]['score']['fullTime']['homeTeam'])
        aw_score_lst.append(an['matches'][i]['score']['fullTime']['awayTeam'])
        status_lst.append(an['matches'][i]['status'])
        date_lst.append(an['matches'][i]['utcDate'])
        
    league_season_df=pd.DataFrame([
                                match_id_lst,
                                home_lst,
                                away_lst,
                                h_score_lst,
                                aw_score_lst,
                                status_lst,
                                date_lst
                                    ]).T

    league_season_df.columns=['id', 'home', 'away', 'gh', 'ga', 'status', 'date']
    
    return league_season_df

In [11]:
for id in tournaments_table['ID'].values[125:132]:
    try:
        an=get_league_data(id)
        league_season_df=get_league_match(an)
        print(tournaments_table[tournaments_table['ID']==id]['Name'].values[0])
    except:
        print(id, 'no acces')

403
2083 no acces
403
2080 no acces
200
Copa Libertadores
403
2081 no acces
403
2082 no acces
403
2079 no acces
403
2077 no acces


In [12]:
len(league_season_df)

30

In [13]:
dl.get_data("""

SELECT *
FROM footbal_data_temp1
limit 5

""")

,match_id,home,away,date,gh,ga,full_time,level,competition,source
0,fdr_0,1 FC Frankfurt Oder,Ruda Hvzda Brno,1960-07-31,2,1,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
1,fdr_1,Ruda Hvzda Brno,1 FC Frankfurt Oder,1960-08-11,2,0,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv
2,fdr_2,Rangers FC,Ferencvarosi Tc,1960-09-28,4,2,F,international,UEFA CW,file:_data/football-data/data/results/uefa-cw.csv


In [19]:
df_results_to_sql[0:3]

,match_id,home,away,date,gh,ga,full_time,level,competition,source
0,api_339896,Montevideo City Torque,Barcelona SC,2022-02-09T00:30:00Z,1,1,F,Unknown,Unknown,api:_http://api.football-data.org/v2/competitions
1,api_339895,Deportivo Lara,Club Bolívar,2022-02-09T22:15:00Z,2,3,F,Unknown,Unknown,api:_http://api.football-data.org/v2/competitions
2,api_339894,CD Universidad César Vallejo,Olimpia Asuncion,2022-02-10T00:30:00Z,0,1,F,Unknown,Unknown,api:_http://api.football-data.org/v2/competitions


In [18]:
league_season_df['match_id'] = 'api_'+league_season_df['id'].astype('str')
league_season_df['full_time'] = 'F'
league_season_df['level'] = 'Unknown'
league_season_df['competition'] = 'Unknown'
league_season_df['source'] = 'api:_'+url

columns_to_sql = [
       'match_id',
       'home', 'away', 'date', 'gh', 'ga', 'full_time', 
       'level', 'competition',
       'source'
            ]

df_results_to_sql = league_season_df[columns_to_sql] 

In [20]:
# to do - datetime format check both
# level and competition vlookup
# teams levenstein

In [8]:
dl.get_data("""

SELECT *
FROM footbal_data
limit 4

""")

,index,home,away,date,gh,ga,full_time,competition,home_ident,away_ident,home_country,away_country,home_code,away_code,home_continent,away_continent,continent,level,file,match_id
0,0,Victoria Kings,Rosignol United,2009-09-06,1,2,F,guyana,Victoria Kings (Guyana),Rosignol United (Guyana),guyana,guyana,GY,GY,South America,South America,South America,national,data/football-data/data/results/guyana.csv,0
1,1,Camptown,Alpha United,2009-09-07,1,3,F,guyana,Camptown (Guyana),Alpha United (Guyana),guyana,guyana,GY,GY,South America,South America,South America,national,data/football-data/data/results/guyana.csv,1
2,2,Rosignol United,Buxton United,2009-09-13,2,0,F,guyana,Rosignol United (Guyana),Buxton United (Guyana),guyana,guyana,GY,GY,South America,South America,South America,national,data/football-data/data/results/guyana.csv,2
3,3,Seawall,Alpha United,2009-09-13,1,1,F,guyana,Seawall (Guyana),Alpha United (Guyana),guyana,guyana,GY,GY,South America,South America,South America,national,data/football-data/data/results/guyana.csv,3


In [12]:
league_season_df

,id,home,away,gh,ga,status,date
0,339896,Montevideo City Torque,Barcelona SC,1,1,FINISHED,2022-02-09T00:30:00Z
1,339895,Deportivo Lara,Club Bolívar,2,3,FINISHED,2022-02-09T22:15:00Z
2,339894,CD Universidad César Vallejo,Olimpia Asuncion,0,1,FINISHED,2022-02-10T00:30:00Z
3,339893,Barcelona SC,Montevideo City Torque,8,7,FINISHED,2022-02-16T00:30:00Z
4,339892,Club Bolívar,Deportivo Lara,4,0,FINISHED,2022-02-16T22:15:00Z
5,339891,Olimpia Asuncion,CD Universidad César Vallejo,2,0,FINISHED,2022-02-17T00:30:00Z
6,339887,CD Everton,Monagas SC,3,0,FINISHED,2022-02-22T22:15:00Z
7,339885,Millonarios FC,Fluminense FC,1,2,FINISHED,2022-02-23T00:30:00Z
8,339886,Club Plaza Colonia,Club The Strongest,2,0,FINISHED,2022-02-23T00:30:00Z
9,339884,América FC,Club Guaraní,0,1,FINISHED,2022-02-23T22:15:00Z


In [67]:
import Levenshtein as lev
api_team_lst=league_season_df['home'].value_counts().index

for team in api_team_lst:
    distance_lst=[]
    for ch in spain_check_lst:
        d=lev.distance(team, ch)
        distance_lst.append(d)
    print(min(distance_lst))
    ind=np.argmin(distance_lst)
    print(team, '-', spain_check_lst[ind])

9
Deportivo Alavés - Deportivo La Coruna
0
Villarreal CF - Villarreal CF
0
Getafe CF - Getafe CF
1
Levante UD - Levante Ud
0
FC Barcelona - FC Barcelona
7
RCD Espanyol de Barcelona - Espanyol Barcelona
9
Real Betis Balompié - Real Betis
0
CA Osasuna - CA Osasuna
0
Elche CF - Elche CF
3
Real Madrid CF - Real Madrid
9
Club Atlético de Madrid - Atletico Madrid
10
Real Sociedad de Fútbol - Real Sociedad
0
Valencia CF - Valencia CF
5
Athletic Club - Athletic Bilbao
0
RCD Mallorca - RCD Mallorca
0
Granada CF - Granada CF
1
Cádiz CF - Cadiz CF
0
Sevilla FC - Sevilla FC
10
Rayo Vallecano de Madrid - Rayo Vallecano
6
RC Celta de Vigo - Celta Vigo


In [57]:
spain_check_lst=['FC Barcelona', 'Real Madrid', 'Valencia CF', 'Athletic Bilbao',
       'Atletico Madrid', 'Espanyol Barcelona', 'Sevilla FC', 'Real Sociedad',
       'Real Zaragoza', 'Real Betis', 'Celta Vigo', 'Deportivo La Coruna',
       'Real Valladolid', 'Sporting Gijon', 'Racing Santander', 'CA Osasuna',
       'Malaga CF', 'Real Oviedo', 'Ud Las Palmas', 'RCD Mallorca',
       'Villarreal CF', 'Granada CF', 'Rayo Vallecano', 'Elche CF',
       'Hercules CF', 'Real Murcia', 'Getafe CF', 'CD Tenerife', 'Levante Ud',
       'Cd Alaves', 'Cadiz CF', 'Ce Sabadell', 'Ud Salamanca', 'CD Logrones',
       'Cd Castellon', 'Burgos CF', 'Cordoba CF', 'Albacete', 'Ud Almeria',
       'Sd Eibar', 'Recreativo Huelva', 'Pontevedra CF', 'Sd Compostela',
       'CD Numancia', 'Arenas De Getxo', 'Gimnastic', 'CD Leganes',
       'Cd Alcoyano', 'Real Jaen', 'CF Extremadura', 'Merida Ad', 'Girona FC',
       'Real Union', 'Ad Almeria', 'Ce Europa', 'Xerez Cd', 'SD Huesca',
       'Lleida Esportiu', 'Atletico Tetuan', 'CD Condal', 'CD Leonesa',
       'Ue Lleida', 'Real Madrid Castilla']